In [1]:
A = open("C:/Users/Admin/Downloads/sherlock.txt",encoding="UTF-8")

In [2]:
data = A.read()

In [3]:
Q = []
import re
for i in data.split(" "):
    w = re.sub("[^a-zA-Z]","",i)
    w = w.upper()
    if(w!=""):
        Q.append(w)

In [4]:
Q = Q[0:1000]

In [26]:
Q

['PROJECT',
 'GUTENBERGS',
 'THE',
 'ADVENTURES',
 'OF',
 'SHERLOCK',
 'HOLMES',
 'BY',
 'ARTHUR',
 'CONAN',
 'DOYLETHIS',
 'EBOOK',
 'IS',
 'FOR',
 'THE',
 'USE',
 'OF',
 'ANYONE',
 'ANYWHERE',
 'AT',
 'NO',
 'COST',
 'AND',
 'WITHALMOST',
 'NO',
 'RESTRICTIONS',
 'WHATSOEVER',
 'YOU',
 'MAY',
 'COPY',
 'IT',
 'GIVE',
 'IT',
 'AWAY',
 'ORREUSE',
 'IT',
 'UNDER',
 'THE',
 'TERMS',
 'OF',
 'THE',
 'PROJECT',
 'GUTENBERG',
 'LICENSE',
 'INCLUDEDWITH',
 'THIS',
 'EBOOK',
 'OR',
 'ONLINE',
 'AT',
 'WWWGUTENBERGNETTITLE',
 'THE',
 'ADVENTURES',
 'OF',
 'SHERLOCK',
 'HOLMESAUTHOR',
 'ARTHUR',
 'CONAN',
 'DOYLERELEASE',
 'DATE',
 'NOVEMBER',
 'EBOOK',
 'LAST',
 'UPDATED',
 'MAY',
 'LANGUAGE',
 'ENGLISHCHARACTER',
 'SET',
 'ENCODING',
 'UTF',
 'START',
 'OF',
 'THIS',
 'PROJECT',
 'GUTENBERG',
 'EBOOK',
 'THE',
 'ADVENTURES',
 'OF',
 'SHERLOCK',
 'HOLMES',
 'PRODUCED',
 'BY',
 'AN',
 'ANONYMOUS',
 'PROJECT',
 'GUTENBERG',
 'VOLUNTEER',
 'AND',
 'JOSE',
 'MENENDEZCOVERTHE',
 'ADVENTURES',
 'OF'

In [5]:
from numpy import unique
unique_words = unique(Q)

In [6]:
Xstr = []
Ystr = []
for i in range(4,len(Q),1):
    w1 = Q[i-4]
    w2 = Q[i-3]
    w3 = Q[i-2]
    w4 = Q[i-1]
    wy = Q[i]
    line = w1 + " " + w2 + " " + w3 + " " + w4
    Xstr.append(line)
    Ystr.append(wy)

In [7]:
lines = len(Xstr)
n_uniques = len(unique_words)
seq = 4

In [8]:
lines

996

In [9]:
n_uniques

515

In [10]:
from numpy import zeros
X_arr = zeros((lines,n_uniques,seq),dtype="uint8") 
Y_arr = zeros((lines,n_uniques),dtype="uint8")

In [11]:
line_enum = list(enumerate(Xstr))

In [12]:
pos_finder = {}
for i,j in list(enumerate(unique_words)):
    pos_finder[j] = i

In [13]:
for line_no,line in line_enum:
    for seq,word in enumerate(line.split(" ")):
        X_arr[line_no,pos_finder[word],seq] = 1

In [14]:
pos_finder["SHERLOCK"]

395

In [16]:
for i,j in enumerate(Ystr):
    pos = pos_finder[j]
    Y_arr[i,pos]=1

In [17]:
X_arr.shape

(996, 515, 4)

In [19]:
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.activations import softmax
from keras.layers.core import Activation

nn = Sequential()
nn.add(LSTM(200,input_shape=(515,4)))
nn.add(Dense(515))
nn.add(Activation("softmax"))

In [20]:
nn.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")
model = nn.fit(X_arr,Y_arr,epochs=2,validation_split=0.01)

Epoch 1/2
31/31 [==============================] - 17s 504ms/step - loss: 6.0522 - accuracy: 0.0527 - val_loss: 6.8871 - val_accuracy: 0.0000e+00
Epoch 2/2
31/31 [==============================] - 16s 513ms/step - loss: 5.7269 - accuracy: 0.0578 - val_loss: 7.3911 - val_accuracy: 0.0000e+00


In [38]:
q = input("Enter sentence:(4 words)")
q = q.upper()

Enter sentence:(4 words)of sherlock holmes by


In [39]:
q

'OF SHERLOCK HOLMES BY'

In [40]:
inp = zeros((2,515,4),dtype="bool")

for word_no,word in enumerate(q.split(" ")):
    inp[1,pos_finder[word],word_no]=1

In [45]:
import pandas as pd
B = pd.DataFrame(zip(unique_words,list(nn.predict(inp)[1])),columns=["Word","Prob"])
x = B.Prob.max()

In [46]:
B[B.Prob == x].Word.values[0]

'THE'